This notebook to test the results of a trained segmentation model. This notebook will apply the segmentation model to any GLIMS_ID and produce a list of segmentations.

In [ ]:
import sys
import os
sys.path.insert(0, os.path.join(os.path.expanduser("~"),"Desktop","projects", "GlacierView", "src","segmentation","helpers"))

from tensorflow import keras
import numpy as np
import read
import preprocess
import matplotlib.pyplot as plt
from skimage.filters import gaussian
from datetime import datetime
import imageio.v2 as imageio


In [ ]:
#define inputs

#glims_id = 'G006819E45785N' #lex blanche
glims_id = 'G007026E45991N' #trient
#glims_id = 'G086519E27919N'

data_label = "full_time_series"
ee_data_dir = os.path.join(os.path.expanduser("~"),
    "Desktop", "projects","GlacierView", "src", "earth_engine","data","ee_landing_zone",data_label)
dem_data_dir = os.path.join(os.path.expanduser("~"),
    "Desktop", "projects","GlacierView", "src", "earth_engine","data","ee_landing_zone","full_time_series")                         
landsat_dir = os.path.join(ee_data_dir, "landsat")
dem_dir = os.path.join(dem_data_dir, "dems")

glacier_dir = os.path.join(landsat_dir, glims_id)
dem_path = os.path.join(dem_dir, f"{glims_id}_NASADEM.tif")

In [ ]:
common_bands = ['blue','green','red','nir','swir','thermal','swir_2']
dim = (128,128)

In [ ]:
#read and preprocess images

images = read.get_rasters(glacier_dir)

sample_image_key = list(images.keys())[0]
print(f"Before preprocessing single image shape: {images[sample_image_key].shape}")
images = preprocess.get_common_bands(images,common_bands)
images = preprocess.normalize_rasters(images)
images = preprocess.resize_rasters(images,dim)
print(f"After preprocessing single image shape: {images[sample_image_key].shape}")

In [ ]:
#read and preprocess dems

dem = read.get_dem(dem_path)

dem_key = list(dem.keys())[0]
print(f"Before preprocessing single dem shape: {dem[sample_dem_key].shape}")
dem = preprocess.resize_rasters(dem, dim)
dem = preprocess.normalize_rasters(dem)
print(f"After preprocessing single dem shape: {dem[sample_dem_key].shape}")

In [ ]:
combined_images_and_dems = [np.concatenate((images[file_name], dem[f"{glims_id}_NASADEM.tif"]),axis = 2) for file_name in sorted(images.keys())]

In [ ]:
#match images with labels
X = np.stack(combined_images_and_dems)
X_smoothed = gaussian(X, sigma = [20,0,0,0], mode = 'reflect')    0.94 0.92 0.99 0.83 0.84
image_file_names_ordered = sorted(images.keys())
image_dates = [datetime.strptime(f.split("_")[1],'%Y-%m-%d') for f in image_file_names_ordered]

In [ ]:
#apply the saved model
saved_models_dir = os.path.join(os.path.expanduser("~"),"Desktop","projects", "GlacierView", "src","segmentation","saved_models")
saved_model_path = os.path.join(saved_models_dir, "re_ni_sw_de_v1.h5")

model = keras.models.load_model(saved_model_path, compile = False)
#predictions = model.predict(X_smoothed)

In [ ]:
#create GIF
gif_creation_dir = os.path.join(os.path.expanduser("~"), "Desktop", "projects", "GlacierView", "src", "segmentation","tmp","gif_creation")
gif_output_dir = os.path.join(os.path.expanduser("~"), "Desktop", "projects", "GlacierView", "src", "segmentation", "gifs")

for f in os.listdir(gif_creation_dir):
    os.remove(os.path.join(gif_creation_dir, f))

for i in range(X_smoothed.shape[0]):
    if i%5 == 0: #ignores 80% of images to run faster
        fig, axs = plt.subplots(1, figsize=(10,10)) ##update the number of suplots to equal the number of layers you want to display
        fig.suptitle(image_file_names_ordered[i])
        axs.imshow((X_smoothed[i,:,:,:][:,:,[2,1,0]]))
        #axs[1].imshow(predictions[i,:,:,:])

        plt.savefig(os.path.join(gif_creation_dir,f'{image_file_names_ordered[i]}.png'), dpi = 100)
        plt.show()

with imageio.get_writer(os.path.join(gif_output_dir,f"{glims_id}.gif"), mode='I') as writer:
    for filename in sorted(os.listdir(gif_creation_dir)):
        image = imageio.imread(os.path.join(gif_creation_dir,filename))
        writer.append_data(image)


In [ ]:
#generate and save surface area time_series

ts_output_dir = os.path.join(os.path.expanduser("~"), "Desktop", "projects", "GlacierView", "src", "segmentation", "surface_area_time_series")
total_areas = []
for prediction in predictions:
    total_areas.append(np.sqrt(np.sum(prediction)))
plt.plot(image_dates, total_areas)
plt.title("Estimated Surface Area (no units)")
plt.savefig(os.path.join(ts_output_dir, f"{glims_id}.png"))

plt.show()